In [2]:
import cv2
import mediapipe as mp
from transformers import pipeline, AutoTokenizer, AutoModelForSequenceClassification

# Step 1: Setup Hugging Face sentiment analysis model
model_id = "distilbert-base-uncased-finetuned-sst-2-english"
tokenizer = AutoTokenizer.from_pretrained(model_id)
base_model = AutoModelForSequenceClassification.from_pretrained(model_id)
hf_pipeline = pipeline("sentiment-analysis", model=base_model, tokenizer=tokenizer)

# Step 2: Setup MediaPipe for Hand Gesture Recognition
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(min_detection_confidence=0.5, min_tracking_confidence=0.5)
mp_drawing = mp.solutions.drawing_utils

# Start camera capture
cap = cv2.VideoCapture(0)

# Check if the camera is opened
if not cap.isOpened():
    print("Error: Could not open video stream.")
else:
    while True:
        ret, frame = cap.read()
        
        if not ret:
            print("Error: Failed to capture image from camera.")
            break
        
        frame = cv2.flip(frame, 1)  # Flip frame to mirror for better hand gestures
        framergb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)  # Convert the frame to RGB
        result = hands.process(framergb)  # Detect hands

        if result.multi_hand_landmarks:
            for hand_landmarks in result.multi_hand_landmarks:
                mp_drawing.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)

                thumb_tip = hand_landmarks.landmark[4]  # Tip of thumb
                thumb_base = hand_landmarks.landmark[2]  # Base of thumb

                # Check thumb position to determine "Like" or "Dislike"
                if thumb_tip.y < thumb_base.y:
                    gesture = "LIKE"
                    color = (0, 255, 0)  # Green for LIKE
                    sentiment_text = "I love this!"  # Example text for LIKE
                else:
                    gesture = "DISLIKE"
                    color = (0, 0, 255)  # Red for DISLIKE
                    sentiment_text = "I hate this!"  # Example text for DISLIKE

                # Display the gesture on screen
                cv2.putText(frame, gesture, (50, 100), cv2.FONT_HERSHEY_SIMPLEX, 1.5, color, 3)

                # Step 3: Analyze sentiment with Hugging Face model
                sentiment_result = hf_pipeline(sentiment_text)
                sentiment_label = sentiment_result[0]['label']
                sentiment_score = sentiment_result[0]['score']

                # Display the sentiment on the screen
                cv2.putText(frame, f"Sentiment: {sentiment_label} ({sentiment_score:.2f})", 
                            (50, 200), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)

        # Show the resulting frame
        cv2.imshow("Hand Gesture Recognition & Sentiment Analysis", frame)

        # Exit on 'q' key
        if cv2.waitKey(1) & 0xFF == ord("q"):
            break

    cap.release()
    cv2.destroyAllWindows()